<a href="https://colab.research.google.com/github/esephaneli/cryptoNexus/blob/main/cryptoNexus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Bu notebook, projenin konseptini göstermek için sadeleştirilmiş bir demodur; production servis kodları paylaşılmamıştır.**


Kurulum + Import + Yardımcılar

In [ ]:
!pip -q install requests python-dotenv openai

import os, math, json
import requests
from datetime import datetime, timezone

CG_BASE = "https://api.coingecko.com/api/v3"

HTTP Helper + CoinGecko: Top20 & Market Chart

In [ ]:
def http_get(url, params=None, timeout=20):
    r = requests.get(url, params=params, timeout=timeout, headers={"accept": "application/json"})
    r.raise_for_status()
    return r.json()

def top20(vs="usd"):
    params = {
        "vs_currency": vs,
        "order": "market_cap_desc",
        "per_page": 20,
        "page": 1,
        "sparkline": "false",
        "price_change_percentage": "1h,24h,7d",
    }
    return http_get(f"{CG_BASE}/coins/markets", params=params)

def market_chart(coin_id, days=30, vs="usd"):
    params = {"vs_currency": vs, "days": int(days)}
    return http_get(f"{CG_BASE}/coins/{coin_id}/market_chart", params=params)


#Metriklerin hesaplanması SMA / RSI / Volatility / Return

In [ ]:
def pct_change(a, b):
    if b == 0:
        return None
    return (a - b) / b

def mean(xs):
    xs = [x for x in xs if x is not None]
    return sum(xs) / len(xs) if xs else None

def stdev(xs):
    xs = [x for x in xs if x is not None]
    if len(xs) < 2:
        return None
    m = mean(xs)
    var = sum((x - m) ** 2 for x in xs) / (len(xs) - 1)
    return math.sqrt(var)

def sma(series, window):
    if len(series) < window:
        return None
    return sum(series[-window:]) / window

def rsi(series, period=14):
    if len(series) < period + 1:
        return None
    gains, losses = 0.0, 0.0
    for i in range(-period, 0):
        diff = series[i] - series[i-1]
        if diff >= 0:
            gains += diff
        else:
            losses += abs(diff)
    if losses == 0:
        return 100.0
    rs = (gains/period) / (losses/period)
    return 100.0 - (100.0/(1.0+rs))


Pipeline : coin seç -> payload üret

In [ ]:
def build_payload(coin_row, chart, days, vs):
    prices = chart.get("prices", [])
    vols = chart.get("total_volumes", [])
    if not prices:
        raise ValueError("Fiyat verisi boş geldi.")

    price_series = [float(v) for _, v in prices]
    vol_series = [float(v) for _, v in vols] if vols else []

    first, last = price_series[0], price_series[-1]
    period_return = pct_change(last, first)

    daily_returns = [pct_change(price_series[i], price_series[i-1]) for i in range(1, len(price_series))]
    vol_daily = stdev(daily_returns)

    payload = {
        "meta": {
            "name": coin_row.get("name"),
            "symbol": coin_row.get("symbol"),
            "coin_id": coin_row.get("id"),
            "days": int(days),
            "vs_currency": vs,
            "generated_at_utc": datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S"),
        },
        "snapshot": {
            "current_price": coin_row.get("current_price"),
            "market_cap": coin_row.get("market_cap"),
            "total_volume": coin_row.get("total_volume"),
            "high_24h": coin_row.get("high_24h"),
            "low_24h": coin_row.get("low_24h"),
            "price_change_24h": coin_row.get("price_change_percentage_24h"),
        },
        "computed": {
            "return_period": period_return,
            "volatility_daily_stdev": vol_daily,
            "sma_7": sma(price_series, 7),
            "sma_14": sma(price_series, 14),
            "sma_30": sma(price_series, 30),
            "rsi_14": rsi(price_series, 14),
        },
        "series": {
            "prices": price_series,
            "volumes": vol_series,
        }
    }
    return payload


#OpenAi Katmanı

In [ ]:

from openai import OpenAI

SYSTEM = """Sen bir kripto veri analisti asistanısın.
.....
"""

def generate_commentary(payload):
    api_key = os.getenv("OPENAI_API_KEY", "").strip()
    if not api_key:
        return "AI yorum kapalı (OPENAI_API_KEY yok). Yatırım tavsiyesi değildir."

    client = OpenAI(api_key=api_key)

    small = {"meta": payload["meta"], "snapshot": payload["snapshot"], "computed": payload["computed"]}

    user = f"JSON:\n{json.dumps(small, ensure_ascii=False, indent=2)}\n\nKısa analiz yaz."
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"system","content":SYSTEM},{"role":"user","content":user}],
        temperature=0.5,
        max_tokens=550,
    )
    return resp.choices[0].message.content


#Çalıştırılan Uygulama Top20 çek -> seç -> analiz çıkart

In [ ]:
VS = "usd"
DAYS = 30

coins = top20(vs=VS)

for i, c in enumerate(coins, 1):
    print(f"{i:>2}. {c['name']} ({c['symbol'].upper()}) | ${c['current_price']} | id={c['id']}")

pick = int(input("\nSeç (1-20): ").strip())
coin = coins[pick - 1]

chart = market_chart(coin["id"], days=DAYS, vs=VS)
payload = build_payload(coin, chart, DAYS, VS)

comp = payload["computed"]
def pct(x): return "N/A" if x is None else f"{x*100:.2f}%"

print("\n----- ANALYSIS -----")
print(f"Coin: {payload['meta']['name']} ({payload['meta']['symbol'].upper()})")
print(f"Return ({DAYS}d): {pct(comp['return_period'])}")
print(f"Vol (daily stdev): {comp['volatility_daily_stdev']}")
print(f"SMA 7/14/30: {comp['sma_7']} / {comp['sma_14']} / {comp['sma_30']}")
print(f"RSI(14): {comp['rsi_14']}")
print("--------------------")



#Akış diyagramı

In [ ]:
"""
CRYPTO NEXUS (Concept Demo) - PIPELINE

[User selects coin]
        |
        v
[Fetch Top 20] ----> (Dropdown/List)
        |
        v
[Fetch Market Chart (N days)]
        |
        v
[Compute Indicators]
  - SMA(7/14/30)
  - RSI(14)
  - Volatility (daily stdev)
  - Period Return
        |
        v
[LLM Commentary Layer - OPTIONAL]
  - Reads ONLY computed metrics (not raw series)
  - Explains trend/momentum/risk
  - NOT a decision-maker
        |
        v
[Output]
  - Numeric summary
  - Optional AI commentary
  - (Optional) downloadable report in full project
"""


#Burada yapılan çalışma öğrenim amaçlıdır. Yatırım Tavsiyesi Değildir !